# import data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
from __future__ import print_function
import inflect
import string
import spacy
import nltk
import re
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud
from textblob import TextBlob
import contractions 

In [ ]:
#display the full dataframe for all cells
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# display(df)

In [ ]:
PROJ_ROOT = os.pardir
print(PROJ_ROOT)
import sys
src_dir = os.path.join(PROJ_ROOT, "src")
sys.path.append(src_dir)

In [ ]:
from config import TRAIN_FILE_PATH, TEST_FILE_PATH
from features.build_features import read_tsv_file
train = read_tsv_file(TRAIN_FILE_PATH)
test = read_tsv_file(TEST_FILE_PATH)
print(train.shape, test.shape)

join the two datasets 

In [ ]:
df = pd.concat([train, test], ignore_index = True)
df.sample(2)

# data wrangling

2.1 Know the basics of the datasets:

1. shape of dataset
2. data type
3. data distribution
4. missing value and the way to handle the missing value
5. any duplicates
6. any incorrect or manipulated data?

In [ ]:
print(df.shape)
print(df.info())
print(df.isnull().sum())

there are 1194 missing values in "condition", also the data type for rating should be int instead of float, the date should change to time.

In [ ]:
df['rating'] = df['rating'].astype('int')
df['date'] = pd.to_datetime(df['date'])

In [ ]:
print(df.rating.unique())
print(df.describe())

 There are 10 unique rating from 1 to 10.  The average rating is 6.99, with the 25% to 75% in 5 to 10, suggesing rating is skewed. 
 the mean "usefulCount" is 28 while the max can reach to 1291 suggesting the usefulCount is widespread. 

Data cleaning 

missing values

In [ ]:
missing_values = df["condition"].isna()
df[missing_values].head()

In [ ]:
missing_value_ratio = df.isna().sum()/len(df)*100
print(round(missing_value_ratio,2))

only 0.56% missing values, and the review of  it is safe to drop it.

In [ ]:
df = df.dropna()
print(df.shape)
# check duplicate of data
print (df.duplicated(subset =["review"]).sum())
print (df.duplicated(subset =["review","condition","rating","usefulCount"]).sum())

In [ ]:
duplicate_rows = df[df.duplicated(subset=["review","condition","rating","usefulCount"])]
duplicate_rows.head()

There are 85420 duplicated in "reviews", for each pair of duplicates, they share the same "condition", while varied in "drugname". Therefore, the duplicate data will be dropped.

In [ ]:
df = df.drop_duplicates(subset=["review","condition","rating","usefulCount"], keep="first")
df.shape

Cleaning "condition" column

In [ ]:
df.condition.unique()

some conditions list are comments which can't represent the real conditions, and should be removed form the dataset. Also, some typos such as "Cance", "Disorde", and incomplete information (e.g "eve", which should be "fever")

In [ ]:
#remove the comments in conditions
condition_mask = df.condition.str.contains("users found this comment helpful")
df=df[~condition_mask]
df.condition.unique()

In [ ]:
repl_dict1 = {
    'emale Infertility': 'Female Infertility',
    'atigue':'Fatigue',
    'Not Listed / Othe': 'Not Listed Other',
    'moterol)':'Formoterol Mometasone',
    't Pac with Cyclobenzaprine (cyclobenzaprine)':
    'Comfort Pac with Cyclobenzaprine',
    'zen Shoulde': 'Frozen Shoulder',
    'mis': 'Mist',
    'tic (mycophenolic acid)': 'Mycophenolic Acid',
    'ailure to Thrive': 'Failure To Thrive',
    'm Pain Disorde': 'Pain Disorder',
    'mist (': 'Mist',
    'me': 'Mist',
    'lic Acid Deficiency': 'Folic Acid Deficiency',
    'min / saxagliptin)': 'Metformin Saxagliptin',
    'ge HCT (amlodipine / hydrochlorothiazide / valsartan)':
    'Amlodipine Hydrochlorothiazide Valsartan',
    'moterol / mometasone)':'Formoterol Mometasone',
    'eve':'Fever',
    'mance Anxiety':'Performance Anxiety',
    'min)':'Metformin Saxagliptin',
    'ge (amlodipine / valsartan)':'Amlodipine Valsartan',
    'min / rosiglitazone)':'Metformin Rosiglitazone',
    'llicular Lymphoma':'Follicular Lymphoma',
    'min / pioglitazone)':'Metformin Pioglitazone',
    'Pe':"Performance Anxiety",
    't Care':'Urgent Care',
    'llicle Stimulation':'Follicle Stimulation',
}

df.replace({"condition": repl_dict1},inplace=True)
repl_dict = {" Disorde$":' Disorder', 'Cance$': 'Cancer',' Cance$': ' Cancer',' Tum$':' Tumor', ' Feve$':' Fever',' Ulce$': ' Ulcer', ' Bladde$':' Bladder'}
df.replace({"condition": repl_dict},regex= True,inplace=True)
df.condition.unique()

In [ ]:
# import string
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text.lower()

In [ ]:
df["condition"] = df['condition'].apply(remove_punctuations)
df.condition.unique()

In [ ]:
rating_by_condition = df.groupby('condition').agg({'rating':'count'}).sort_values(by ='rating',ascending = False)
rating_by_condition = rating_by_condition.nlargest(20, 'rating')
# visualize the results
plt.figure(figsize =(10,6))
rating_by_condition.plot(kind='bar')
plt.title('sum of rating by conditions')
plt.xlabel('conditions')
plt.ylabel('count of rating')
plt.show()

The'birth control', 'depression','pain', 'anxiety' are the conditions have most of rating. 

In [ ]:
drugName_list = df.drugName.unique().tolist()
print(len(drugName_list))
print(drugName_list)

In [ ]:
# check the most  count drugNme 
plt.figure(figsize = (10,8))
top_drugName = df['drugName'].value_counts(ascending = False).nlargest(20)
top_drugName.plot(kind ='bar')
plt.title('top 20 counts of drugName')
plt.xlabel('Drug Name')
plt.ylabel('counts')
plt.show()

what are the top 10 drugs used in birth contorl?

In [ ]:
d1 =df[df['condition'] =='birth control']['drugName'].value_counts()[0:10]
d1.plot(kind = 'bar')
plt.title('top 10 drugname in birth control ')
plt.xlabel('Drug Name')
plt.ylabel('counts')
plt.show()

what are the drugs with the highest rating?

In [ ]:
top_rating_drug = df[df['rating'] == 10]['drugName'].value_counts()
#plot the top 10 drug name with rating == 10
top_rating_drug[0:10].plot(kind = 'bar')
plt.title('top 10 drugname with rating =10 ')
plt.xlabel('Drug Name')
plt.ylabel('counts')
plt.show()

In [ ]:
df['rating'].hist(bins=10)
plt.title('histogram of rating in drug review')
plt.xlabel("rating")
plt.ylabel("counts")
plt.show()

In [ ]:
df['rating_category'] =pd.cut(df['rating'], bins = [1,4,7,10],labels = ['negative','neutral','positive'])
print(df['rating_category'].value_counts(normalize = True)*100)

df['rating_category'].hist(bins=10)
plt.title('histogram of rating in drug review')
plt.xlabel("rating")
plt.ylabel("counts")
plt.show()

The rating is not eqully distributed, about 70% positive rating, only 17 % and 13% neutral and negative rating. The data is imbalanced.

visulaize the distribution of 'usefulCount'.

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
sns.histplot(df['usefulCount'], kde=True, bins=20, color='skyblue', edgecolor='black')
plt.title('Kernel Density Estimation with Histogram for Useful Count')
plt.xlabel('Useful Count')
plt.ylabel('Density/Frequency')

plt.show()

what are the trends of reviews over years?

In [ ]:
review_trend = df['date'].dt.year.value_counts()
review_trend.sort_index()
sns.barplot(x= review_trend.index, y =review_trend.values,color = 'b').set_title('number of review per year')
plt.xlabel('years')
plt.show()

## cleaning "review" and EDA

To clean 'review', several steps will performed in a sequence:
1. remove_special_characters, especial html encorded one.
2. convert number to words
3. expand contraction
4. remove whitespace
5. remove stopwords
6. stem and lemmerlize word.

In [ ]:
def remove_special_characters(text):
    text = text.lower()
    text = text.replace('&#039', '').replace('\n','').replace('\r', '')
    text = text.replace(r'[^\w\d\s]',' ')
    pattern = re.compile(r'[^a-zA-z0-9\s]+')
    cleaned_text = re.sub(pattern, '', str(text))
    cleaned_text =' '.join(word.strip() for word in cleaned_text.split())
    return cleaned_text                

In [ ]:
# import inflect
p = inflect.engine()
# convert number into words
def convert_number(text):
    temp_text = text.split()
    new_text = []
    for word in temp_text:
        if word.isdigit():
            temp = p.number_to_words(word)
            new_text.append(temp)
        else:
            new_text.append(word)
    temp_text = ' '.join(new_text)
    return temp_text
    

In [ ]:
df['review'] = df['review'].apply(convert_number)

In [ ]:
# import contractions 
def expand_contractions(text):
    cleaned_text = contractions.fix(text)
    return cleaned_text

In [ ]:
def remove_whitespace(text):
    return ' '.join(text.split())

In [ ]:
# import en_core_web_sm
import nltk
from nltk.corpus import stopwords
 
nltk.download('stopwords')
# from nltk.corpus import stopwords
# nlp= spacy.load('en_core_web_sm')
stop_words = set(stopwords.words('english'))
# import nltk
nltk.download('punkt')
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word not in stop_words]
    return  ' '.join(filtered_text)

In [ ]:
df['review_clean'] = df['review'].apply(remove_special_characters)
df['review_clean'] = df['review'].apply(convert_number)
df['review_clean'] = df['review'].apply(expand_contractions)
df['review_clean'] = df['review'].apply(remove_whitespace)
df['review_clean'] = df['review'].apply(remove_stopwords)


In [ ]:
Snow_ball = nltk.stem.SnowballStemmer("english")
df['review_clean'] = df['review_clean'].apply(lambda x: " ".join(Snow_ball.stem(word) for word in x.split()))

In [ ]:

# import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
df['review_clean'] = df['review_clean'].apply(lambda x: " ".join(lemmatizer.lemmatize(word) for word in x.split()))

In [ ]:
df[['review_clean']].head(5)

check the lenth of reviews

In [ ]:
# number of words in each review
df['word_count'] = df['review_clean'].apply(lambda x: len(str(x).split()))

In [ ]:
#average lenth of words
df["mean_word_len"] = df["review_clean"].apply(lambda x: np.mean([len(w) for w in str(x).split()]))

In [ ]:
# unique word in each review
df['unique_word_count'] = df['review_clean'].apply(lambda x: len(set(str(x).split())))

In [ ]:
df.head()

Genuine of rating per review

In [ ]:
from textblob import TextBlob

In [ ]:
#define a function to obtain sentiment values
def sentiment_score(text):
    blob = TextBlob(text)
    return blob.polarity

In [ ]:
def sentiment_subjectivity(text):
    blob = TextBlob(text)
    return blob.subjectivity

In [ ]:
#define a function to label the review in three sentiments: positive, negative and neutral.
def sentiment_label(text):
    blob =TextBlob(text)
    if blob.polarity>0:
        return "positive"
    elif blob.polarity ==0:
        return "neutral"
    else:
        return "negative"
    

In [ ]:
df['sentiment_subjectivity'] = df['review_clean'].apply(sentiment_subjectivity)
df['sentiment_score'] = df['review_clean'].apply(sentiment_score)
df['sentiment_label'] = df['review_clean'].apply(sentiment_label)


In [ ]:
print(df['sentiment_label'].value_counts())
df['sentiment_label'].value_counts().plot(kind ='bar')
plt.show()

In [ ]:
#check the sentiment_subjectivity distribution
sns.kdeplot(df['sentiment_subjectivity'], color='green', fill=True)
plt.title('Kernel Density Estimation of sentiment_subjectivity')
plt.xlabel('sentiment_subjectivity')
plt.ylabel('Density')

# Show the plot
plt.show()

Does the sentiment labels match with the rating?

In [ ]:
sns.lineplot(data = df, x = 'rating', y ='sentiment_subjectivity', hue ='sentiment_label')
plt.title('Sentiment subjectivity by Rating')
plt.xlabel('Rating')
plt.ylabel('Sentiment subjectivity')
# Move the legend to the upper-right corner
plt.legend(loc='upper right')
plt.show()

In [ ]:
sns.lineplot(data = df, x = 'rating', y ='sentiment_score', hue ='sentiment_label')
plt.title('Sentiment score by Rating')
plt.xlabel('Rating')
plt.ylabel('Sentiment score')
# Move the legend to the upper-right corner
plt.legend(loc='upper right')
plt.show()

The plot shows for each type of sentiments, there are widespread of rating. Below give the way to check the genuine rating of reviews.

In [ ]:
df['genuine_positive'] = np.where(
    (df['rating_category'] == 'positive') &
    (df['sentiment_label'] == 'positive') &
    (df['sentiment_subjectivity'] <= 0.3),
    1,
    0
)

In [ ]:
df['genuine_negative'] = np.where(
    (df['rating_category'] == 'negative') &
    (df['sentiment_label'] == 'negative') &
    (df['sentiment_subjectivity'] <= 0.3),
    1,
    0
)

In [ ]:
df['genuine_neutral'] = np.where(
    (df['rating_category'] == 'neutral') &
    (df['sentiment_label'] == 'neutral') &
    (df['sentiment_subjectivity'] <= 0.3),
    1,
    0
)

In [ ]:
df.head()

save data 

In [ ]:
df.to_csv("../data/processed/drug_review_clean.csv",index=False)

Summary

1.The two raw .tsv data were first loaded and concatted into one dataframe, with a shape of (215063, 7). 
2.1194 records missing "condition" and the missing ratio is 0.56%, they were dropped from the dataset.
3.more than 1000 conditions with comments were also removed.
4. The condition col has typos, wrong spelling, and was cleaned accordingly using .apply(dictionary)
5. Over 85000k duplicated in subset"review".The duplicate reviews exhibit variations only in the drug names for the same condition. As a result, only the initial record were retain.
6.Visualize the top conditions with rating, top review drugs for birth control.
7.clean "review" column by several functions including remove_special characters, remove_whitespace, remove_stopwords, expand_contraction, stemming and lemminization. 
8 new columns such as "rating_category",'word_count', 'mean_word_len', 'unique_word_count', 'genuine_positive','genuine_negative','genuine_neutral', 'sentiment_subjectivity','sentiment_score','sentiment_label']
